# Build a Simple LLM

In [1]:
import getpass
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Using Language Model

Initiate Gemini model

In [3]:
if not os.getenv("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [4]:
model.invoke("Hello, world!")

AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--41d3e606-ed3d-41c8-b488-0cec5625ee4d-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

1. Create a list of messages with different conversational roles for each message object.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

2. Pass messages to model with `invoke()`

In [6]:
model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--81643e4b-d299-45af-be62-5ed0e4d71e37-0', usage_metadata={'input_tokens': 9, 'output_tokens': 3, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

Various formats of messages are acceptable. The following are all the same

In [7]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}]) # OpenAI Format

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c69624c3-f799-457f-ab6a-505fa3b19a29-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

Models are Runnable objects: Async and streaming modes of invocation are possible

In [8]:
for token in model.stream(messages):
    print(token.content, end="|")

Ciao|!
|

## Prompt Templates

Basically insert variables into the prompt to give them flexibility

In above example
 - We don't have to always translate from English into Italian
 - We don't always have to translate "hi!"

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

We fill in the above template variables (`language`, `text`) with a dictionary.

Now we can always translate any message from English into any language

In [11]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

A "filled in" prompt has a ChatPromptValue containing everything inside the Prompt Template

In [12]:
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

Get the `messages` attribute directly

In [13]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

Invoke the model with the "filled in" prompt template

In [14]:
response = model.invoke(prompt)
print(response.content)

Ciao!


In [15]:
prompt = prompt_template.invoke({"language": "Chinese (Simplified)", "text": "hi!"})

In [16]:
response = model.invoke(prompt)
print(response.content)

嗨！ (Hāi!)


In [17]:
prompt = prompt_template.invoke({"language": "Italian", "text": "Good morning"})
response = model.invoke(prompt)
print(response.content)

Buongiorno
